In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    #Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

In [7]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [9]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 5s - loss: 3.2623 - val_loss: 3.2319 - 5s/epoch - 5s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2253 - val_loss: 3.2067 - 82ms/epoch - 82ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1964 - val_loss: 3.1845 - 84ms/epoch - 84ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1718 - val_loss: 3.1658 - 81ms/epoch - 81ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.1520 - val_loss: 3.1498 - 83ms/epoch - 83ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1358 - val_loss: 3.1357 - 85ms/epoch - 85ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.1221 - val_loss: 3.1232 - 82ms/epoch - 82ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.1100 - val_loss: 3.1122 - 83ms/epoch - 83ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.0993 - val_loss: 3.1026 - 79ms/epoch - 79ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.0899 - val_loss: 3.0941 - 84ms/epoch - 84ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.0815 - val_loss: 3.0866 - 82ms/epoch - 82ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.0742 - val_loss: 3.0800 - 82ms/epoch - 82ms/step
Epoch 13/2000
1/1 - 0s - loss

1/1 - 0s - loss: 2.7664 - val_loss: 2.8363 - 80ms/epoch - 80ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.7630 - val_loss: 2.8344 - 91ms/epoch - 91ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.7597 - val_loss: 2.8331 - 80ms/epoch - 80ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.7571 - val_loss: 2.8292 - 83ms/epoch - 83ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.7538 - val_loss: 2.8263 - 82ms/epoch - 82ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.7502 - val_loss: 2.8243 - 80ms/epoch - 80ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.7474 - val_loss: 2.8217 - 83ms/epoch - 83ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.7446 - val_loss: 2.8208 - 80ms/epoch - 80ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.7413 - val_loss: 2.8178 - 83ms/epoch - 83ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.7384 - val_loss: 2.8154 - 81ms/epoch - 81ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.7350 - val_loss: 2.8138 - 80ms/epoch - 80ms/step
Epoch 112/2000
1/1 - 0s - loss: 2.7324 - val_loss: 2.8105 - 84ms/epoch - 84ms/step
Epoch 113/2000
1/1 

Epoch 200/2000
1/1 - 0s - loss: 2.5451 - val_loss: 2.6651 - 75ms/epoch - 75ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.5411 - val_loss: 2.6617 - 83ms/epoch - 83ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.5384 - val_loss: 2.6605 - 80ms/epoch - 80ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.5382 - val_loss: 2.6616 - 76ms/epoch - 76ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.5359 - val_loss: 2.6592 - 81ms/epoch - 81ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.5343 - val_loss: 2.6572 - 81ms/epoch - 81ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.5306 - val_loss: 2.6521 - 83ms/epoch - 83ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.5281 - val_loss: 2.6497 - 79ms/epoch - 79ms/step
Epoch 208/2000
1/1 - 0s - loss: 2.5262 - val_loss: 2.6518 - 76ms/epoch - 76ms/step
Epoch 209/2000
1/1 - 0s - loss: 2.5249 - val_loss: 2.6490 - 82ms/epoch - 82ms/step
Epoch 210/2000
1/1 - 0s - loss: 2.5228 - val_loss: 2.6492 - 72ms/epoch - 72ms/step
Epoch 211/2000
1/1 - 0s - loss: 2.5210 - val_loss: 2.6462 - 83ms/epoch - 83ms/step
Epoc

In [10]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [11]:
(np.mean(acc), np.mean(prob))

(0.882, 0.35875502)

In [12]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
encoder_output = word_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 5s - loss: 3.7363 - val_loss: 3.2841 - 5s/epoch - 5s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2838 - val_loss: 3.1190 - 307ms/epoch - 307ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.0946 - val_loss: 3.0484 - 299ms/epoch - 299ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.0104 - val_loss: 3.0134 - 296ms/epoch - 296ms/step
Epoch 5/2000
1/1 - 0s - loss: 2.9706 - val_loss: 2.9909 - 275ms/epoch - 275ms/step
Epoch 6/2000
1/1 - 0s - loss: 2.9489 - val_loss: 2.9801 - 284ms/epoch - 284ms/step
Epoch 7/2000
1/1 - 0s - loss: 2.9388 - val_loss: 2.9659 - 270ms/epoch - 270ms/step
Epoch 8/2000
1/1 - 0s - loss: 2.9226 - val_loss: 2.9514 - 274ms/epoch - 274ms/step
Epoch 9/2000
1/1 - 0s - loss: 2.9036 - val_loss: 2.9434 - 268ms/epoch - 268ms/step
Epoch 10/2000
1/1 - 0s - loss: 2.8898 - val_loss: 2.9388 - 273ms/epoch - 273ms/step
Epoch 11/2000
1/1 - 0s - loss: 2.8788 - val_loss: 2.9307 - 279ms/epoch - 279ms/step
Epoch 12/2000
1/1 - 0s - loss: 2.8632 - val_loss: 2.9181 - 266ms/epoch - 266ms/step
Epoch 1

In [13]:
x_masked_train

array([[14, 19,  1, 14],
       [ 4,  1,  0,  4],
       [14,  1,  5, 14],
       ...,
       [15,  2, 12, 15],
       [17,  4, 14, 17],
       [ 5, 16,  1,  5]])

In [14]:
y_masked_labels_train

array([[19,  1, 14],
       [ 1,  0,  4],
       [ 1,  5, 14],
       ...,
       [ 2, 12, 15],
       [ 4, 14, 17],
       [16,  1,  5]])

In [15]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [16]:
(np.mean(acc), np.mean(prob))

(0.967, 0.73480076)

In [17]:
x_test_subset

array([[ 5,  0,  2,  5],
       [11,  1,  3, 11],
       [16, 11,  3, 16],
       ...,
       [11, 17,  7, 11],
       [14, 10,  8, 14],
       [ 7, 15,  6,  7]])

In [18]:
temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array([5, 0, 2, 5]).reshape(1,len(sentence_number)))
temp = temp[:,-1,:]
temp

array([[1.5243081e-02, 1.6439389e-03, 1.9486870e-04, 1.3398707e-02,
        1.5217075e-01, 7.3369056e-01, 4.5488998e-03, 3.8815632e-03,
        3.1420204e-04, 1.2989054e-02, 2.4941228e-03, 1.7236115e-03,
        2.4614645e-02, 5.1256698e-03, 8.7624453e-03, 2.0610983e-03,
        1.5556569e-03, 3.9935755e-03, 5.7152491e-03, 5.8782739e-03]],
      dtype=float32)

In [19]:
temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array([0, 5, 2, 0]).reshape(1,len(sentence_number)))
temp = temp[:,-1,:]
temp

array([[8.7793839e-01, 3.9893654e-03, 3.1362392e-04, 2.5042465e-03,
        3.1550489e-03, 1.3760317e-04, 4.9131678e-04, 2.6360264e-03,
        1.9013104e-03, 3.9221297e-04, 1.6857702e-02, 4.8561595e-02,
        4.8413621e-03, 1.7939642e-03, 4.3755607e-03, 1.3562806e-03,
        6.2582488e-03, 2.1750858e-02, 1.2489436e-04, 6.2040123e-04]],
      dtype=float32)